# Data Cleansing on scraped data (csv)
* Part 1: Data retrieval with pandas
* Part 2: Data cleansing on Job Titles
* Part 3: Data cleansing on Job Description
  * Split by keywords to separate Role, Requirements and Benefits
* Part 4: Save to csv

## Part 1: Data retrieval with pandas
* Read scraped data (csv) into pandas dataframe
* Data cleaning - word replacement & keyword extraction

In [12]:
import pandas as pd
import numpy as np
import os
file = os.path.join('data','data','20211004-2133-Indeed_detail.csv') #update filepath: data / (job title) / (filename.csv)

df = pd.read_csv(file)

## Part 2: Data Cleasing on Job Titles
1. Replacing common words with Regex
2. Get Job Level and Salary with Regex

In [13]:
df['jobTitle'].unique()[:20] # quick preview on job titles scraped (first 30 [:30])

array(['Enterprise Data - BQuant Enterprise Sales - Hong Kong',
       'Data Entry Clerk (資料輸入員)', 'Data Entry / 資料輸入員',
       'Data Entry Clerk (1-Year Contract)', 'Data Entry',
       'Part-time data entry(6-8/Oct)',
       'Contract Data Entry Clerk (1 Year)', 'Intern, Data Analytics',
       'Data Entry Clerk', 'Acuris - Data Analyst', 'Junior Data Analyst',
       'Part-Time Data Entry/ 資炓輸入員', 'Data Entry Clerk ( Part Time )',
       '2022 Corporate & Investment Bank - Markets Summer Analyst Program - Research - Hong Kong',
       'Admin Assistant',
       'Solution Analyst – Data Integration and Data Governance',
       'Market Research Executive (Welcome Fresh Grad)',
       'Senior Data Scientist – Hong Kong – Intact Lab',
       'Assistant Data Analytics Officer / Data Analytics Assistant',
       '2022 Market Data Analyst (Chinese Speaker) - Hong Kong'],
      dtype=object)

In [14]:
import re

# Data Cleaning with Regex replacement
di = {
        '^.*[dD]ata [eE]ntry.*$': "Data Entry", # if contains data entry -->"Data Entry"
        '^[Bb]usiness [Aa]nalyst[^,]': "Business Analyst", # if starts with business analyst and not follow by comma --> "Business Analyst"
        '[Aa]sia [Pp]acific|APAC':'APAC,', # Standardising common job title words
        'AVP':'Assistant Vice President',
        'VP':'Vice President',
        '[Ff]resh [Gg]raduates|[Ss]enior|[Jj]unior':'', #removal of job levels
        'Hong Kong|pandamall':'', # removal on common job title ignore words
        '[Cc]orporate [Bb]anking|Top Tier Bank in HK|Banking|Bank$|Investment Bank|[Cc]ore [Bb]anking':'',
        '[Ii]nsurance|[Ff]inancial [Ss]ervices':'',
        '\([\w\d ,\-]*\)|URGENT|\!|\-\-|\-$|\u200b|\– part':'',
        '\|.*':'', # remove everything after "|"
        ' for .*':'' # remove everything after "for"
      }
df['jobTitle_cleaned'] = df['jobTitle'].replace(di, regex=True).str.strip().replace({'^,|,$|\–$|\-$|,$':''},regex=True).str.strip()
df[['jobTitle','jobTitle_cleaned']].tail(50)
print("Total {} rows with {} unique values".format(len(df['jobTitle_cleaned']),len(df['jobTitle_cleaned'].unique())))

Total 391 rows with 286 unique values


In [15]:
# extract job level keywords
def get_job_level(title):
    # Standardising Job Levels
    job_levels=[]
    job_levels.append('VP') if bool(re.search('VP|Vice President',title, re.I)) else None
    job_levels.append('AVP') if bool(re.search('AVP|Assistant Vice President',title, re.I)) else None
    job_levels.append('Senior Manager') if bool(re.search('Senior Manager',title, re.I)) else None
    job_levels.append('Assistant Manager') if bool(re.search('Assistant Manager',title, re.I)) else None
    job_levels.append('Manager') if bool(re.search('(?<!Senior )(?<!Assistant )Manager',title, re.I)) else None
    job_levels.append('Senior') if bool(re.search('Senior(?! Manager)',title, re.I)) else None
    job_levels.append('Junior') if bool(re.search('Junior',title, re.I)) else None
    job_levels.append('Trainee') if bool(re.search('Trainee',title, re.I)) else None
    job_levels.append('Graduates') if bool(re.search('Graduates',title, re.I)) else None
    job_levels.append('Intern') if bool(re.search('Intern',title, re.I)) else None

    #tmp = re.findall("avp|Assistant Vice President|VP|Vice President|Senior Manager|Assistant Manager|Manager|senior|junior|graduate|intern|trainee",title, flags=re.I)
    if len(job_levels) > 0:
        return '/'.join(job_levels)
    return 'General'

df['jobLevel'] = df['jobTitle'].apply(get_job_level)

In [16]:
# Extract Salary specified in Job Title

def get_salary(title):
    salary = re.findall('\d+[ ]?(?=k)',title, re.I) # get all groups with number followed by "K"
    if len(salary)>0:
        return int(salary[-1]) # return the last one (usually the upper limit)
    return np.nan

df['salary'] = df['jobTitle'].apply(get_salary)

In [17]:
# Preview on results
tmp = df[['jobTitle','jobTitle_cleaned','jobLevel','salary']]
tmp
#tmp[tmp['jobLevel']=='Senior Manager/Senior']['jobTitle'].to_list()
#tmp['jobLevel'].unique()
tmp[tmp['salary'] > 0]

,jobTitle,jobTitle_cleaned,jobLevel,salary
36,Data Entry Clerk (3 months) HK$13K-HK$15K,Data Entry,General,15.0
140,7x24 Data Centre Operator / Operation Executiv...,7x24 Data Centre Operator / Operation Executive,General,30.0
153,"Data Specialist (4 months, up to 30k)",Data Specialist,General,30.0
166,"Data Specialist (4 months, up to 30k)",Data Specialist,General,30.0
175,Head of Data Migration (HK$70K – 120K),Head of Data Migration (HK$70K – 120K),General,120.0
179,Data Modeler - Bank (60-70k),Data Modeler - Bank,General,70.0
225,BUSINESS ANALYST (AGILE) (UP TO 50K),BUSINESS ANALYST,General,50.0
238,BUSINESS ANALYST (ERP) (UP TO 50K),BUSINESS ANALYST,General,50.0
260,Data Engagement Manager - Bank (60-70k),Data Engagement Manager - Bank,Manager,70.0
282,Business Performance Analyst 40K,Business Performance Analyst 40K,General,40.0


## Part 3: Data Cleansing on Job Descriptions
1. Replacing key words with Regex
2. Splitting into Roles, Requirements and Benefits

In [18]:
df[['jobTitle','jobDescription']]
df['jobDescription'][0:5].to_list()

# Data Cleaning with Regex replacement
di = {
        'the role[ ]?[\n\:·]|your role[ ]?[\n\:·]|Job Scope|trust you to|What You\'ll Do|Job Highlight|Summary|Responsibilities[\n\:·]|Responsible for|What to expect|Job Description|you will work': "!!![Role]",
        'need to have|What You Have|you must have|You have[ ]?[\n\:·]|Your expertise|Requirement[s]?[ ]?[\n\:·]|Requirements(?! )|Job Requirement|Desired Competencies|What we seek|Who You Are|What you\'ll need|About you|Qualifications': "!!![Requirements]", 
        'What You\'ll Enjoy|Benefits|What we offer|we will offer': "!!![Benefits]", 
        'Apply if|Apply Now|Please send|Please contact|Please Provide|Join Us|Closing Statement|Application Method|is one of the leading|about us':"!!![End]"
      }

def replace_jd_keywords(text):
    for k,v in di.items():
        text = re.sub(k, v, text,flags=re.I)
    return text
        
def split_jd(text, key):
    tmp = []
    for i in text.split('!!!'):
        if i.startswith('['+key+']'):
            txt = re.sub('^\['+key+'\][ :,;]{0,5}','', i).strip().replace('\n',' ')
            tmp.append(txt) if len(txt)>0 else None
    
    if (key=='Requirements') & (len(tmp) == 0):
        tmp.append('[NO REQ]')
        for i in text.split('!!!'):
            if (not i.startswith('[')) & len(i.strip())>0:
                tmp.append(i.strip().replace('\n',' '))
        
    return '| '.join(tmp)
                        
df['JD_cleaned'] = df['jobDescription'].apply(replace_jd_keywords)
df['JD_requirements'] = df['JD_cleaned'].apply(lambda x: split_jd(x,'Requirements'))
df['JD_benefits'] = df['JD_cleaned'].apply(lambda x: split_jd(x,'Benefits'))
df['JD_role'] = df['JD_cleaned'].apply(lambda x: split_jd(x,'Role'))
#.replace(di, flags=re.I).str.strip()
#.replace({'^,|,$|\–$|\-$|,$':''},regex=True).str.strip()


df[['jobDescription','JD_cleaned','JD_requirements','JD_benefits','JD_role']]


 




,jobDescription,JD_cleaned,JD_requirements,JD_benefits,JD_role
0,Interested in engaging with top quantitative f...,Interested in engaging with top quantitative f...,"in data science, quantitative investment strat...",,Position our quant platform to prospective cli...
1,Primetech Technology Limited (“Primetech”) is ...,Primetech Technology Limited (“Primetech”) is ...,中學或以上程度 必須懂中文及英文打字 (中、英文打字每分鐘達20字以上) 必須懂電腦基本運用...,,Our clients are has a group of companies in As...
2,YesStyle.com is the first online retailer in A...,YesStyle.com is the first online retailer in A...,HKDSE or above Able to read and understand Si...,"5 Days Work Dynamic, International Cultural &...",Product data processing for merchandise base o...
3,CK Asset Holdings Limited is one of the larges...,CK Asset Holdings Limited is one of the larges...,[NO REQ]| CK Asset Holdings Limited is one of ...,attractive compensation package to the right c...,
4,Clerical work of operation teamData EntryPart-...,Clerical work of operation teamData EntryPart-...,[NO REQ]| Clerical work of operation teamData ...,,
...,...,...,...,...,...
386,We are a well established and listed local com...,We are a well established and listed local com...,Lead a team to provide day-to-day production s...,,Manage the team to deliver and support the Ora...
387,Seamatch Asia Limited is a leading established...,Seamatch Asia Limited is a leading established...,University graduates in IT or related subjects...,,"system development, implementation, maintenanc..."
388,Seamatch Asia Limited is a leading established...,Seamatch Asia Limited is a leading established...,"Degree holder in Computer Science, Information...",,"Banking / Digital / IT project management, inc..."
389,Seamatch Asia Limited is a leading established...,Seamatch Asia Limited is a leading established...,"Higher Diploma or Degree in Computer Science, ...",,Participate in software development and system...


## Part 3: Output to csv

In [19]:
import os

file = os.path.join('output','data','Indeed.csv') #update filepath: output / ((filename.csv)

outdir = os.path.dirname(file)
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
df.to_csv(file)